## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import glob
import scipy.stats
import scipy.signal

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """ Calculates mean absolute errors and confidences
    
    Args:
        data_fl: (string) Path to the signal file
        ref_fl: (string) Path to the reference signal file
        
    Returns:
        (np.array) Mean absolute errors
        (np.array) Confidences
    """
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # load the reference signal
    ground_truth = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1)
    
    errs = []
    confs = []
    
    start_indices, end_indices = get_start_end(len(accx), len(ground_truth))
    
    for i, start in enumerate(start_indices):
        end = end_indices[i]
        ref = ground_truth[i]
        
        # Bandpass filtering the signals
        w_ppg =  bandpass_filter(ppg[start:end])
        w_accx = bandpass_filter(accx[start:end])
        w_accy = bandpass_filter(accy[start:end])
        w_accz = bandpass_filter(accz[start:end])
        
        # Get prediction
        conf, pred = get_predictions(w_ppg, w_accx, w_accy, w_accz)

        errs.append(np.abs(pred - ref))
        confs.append(conf)
    
    errs = np.array(errs)
    confs = np.array(confs)
    # Return mean absolute error and confidence as a 2-tuple of numpy arrays.
    return errs, confs

In [2]:
fs = 125
window_shift = 2
window_length = 10

def get_start_end(sig_len, ref_len):
    """ Returns the start and end indices of a signal """
    n = ref_len if ref_len < sig_len else sig_len
    start = (np.cumsum(np.ones(n) * fs * window_shift) - fs * window_shift).astype(int) # windows shift of 2
    return (start, start + window_length * fs) # windows length of 10

def bandpass_filter(signal, band_pass = (40/60, 240/60), fs = fs):
    """ Performs a bandpass filter on the signal. """
    
    b,a = scipy.signal.butter(3, band_pass, fs=fs, btype= 'bandpass')
    
    # Perform forward and backward digital butterworth filter
    return scipy.signal.filtfilt(b,a,signal)


def filter_signal(signal, band_pass = (40/60, 240/60)):
    """ Performs bandpass filter and zeros outrange frequencies"""
    
def get_predictions(ppg, accx, accy, accz):
    """ Performs estimations on sensor signals
    
        Args:
            ppg : Photoplethysmography (PPG) sensor signal
            accx: Accelerometer sensor signal along x-axis
            accy: Accelerometer sensor signal along y-axis
            accz: Accelerometer sensor signal along z-axis

        Returns:
            The estimated frequency (in BPM) along with its confidence score.
    """
    
    n = len(ppg) * 4
    freqs = np.fft.rfftfreq(n, 1/fs)

    # Zeroing outrange frequencies
    fft = np.abs(np.fft.rfft(ppg, n))
    fft[freqs <= 40/60.0] = 0.0
    fft[freqs >= 240/60.0] = 0.0

    # Calculate the magnitude
    acc_abs = np.sqrt(accx ** 2 + accy ** 2 + accz ** 2)
    
    # Zeroing outrange frequencies
    acc_fft = np.abs(np.fft.rfft(acc_abs, n))
    acc_fft[freqs <= 40/60.0] = 0.0
    acc_fft[freqs >= 240/60.0] = 0.0

    # Find peaks 
    peaks, _ = scipy.signal.find_peaks(fft, height = 0.5*np.max(fft), distance=1)
    max_ppg_fs = freqs[peaks]

    # Find peaks 
    acc_peaks, _ = scipy.signal.find_peaks(acc_fft, height=0.3*np.max(acc_fft), distance=50)
    
    if len(max_ppg_fs) == 0:
        best_fit = freqs[np.argmax(fft)]
        
    elif len(max_ppg_fs) > 1:
        max_ppg_ids = np.argpartition(fft[peaks], -2)[-2:]
        max_ppg_ids = max_ppg_ids[np.argsort(-fft[peaks][max_ppg_ids])]
        max_ppg_fs = max_ppg_fs[max_ppg_ids]
    
        best_fit = max_ppg_fs[0]
        
        ppg_best_distance = (fft[peaks][max_ppg_ids[0]] - fft[peaks][max_ppg_ids[1]]) / 1000
        if len(acc_peaks) > 0 and ppg_best_distance < fft[peaks][max_ppg_ids[0]] * 0.2:
            best_fit = max_ppg_fs[1]
    else:
        best_fit = max_ppg_fs[0]
        
    fs_win = 45  / 60.0
    fs_win_e = (freqs >= best_fit - fs_win) & (freqs <= best_fit + fs_win)
    
    confidence = np.sum(fft[fs_win_e])/np.sum(fft)
    prediction = best_fit * 60
    
    return (confidence, prediction)

In [3]:
Evaluate()

21.583330594281932

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

- **Code Description** - The code is used to test the performance of the algorithm on the training dataset. You can run the `Evaluate` function to start the algorithm.  


- **Data Description**
    - The dataset consist of signals from accelerometer and PPG sensors. 
    - The accelerometer has three channels, each corresponding to a space axis x, y, and z. I use the magnitude of these three channels as distance calculation. 
    - Both signals have noise but combining them would produce good results. 
    - The data is sampled at 125Hz from 12 different subjects.  


- **Algorithm Description** 
    1. The algorithm works as follows:  
        - Performs bandpass filtering on both signals.
        - Applies FFT to both signals.
        - Find the peaks of both signals.
            - If no peaks are found, use the strongest frequency.
            - If one peak is found, use it.
            - If more than one peak is found and the distance between the two frequencies is small, use the second strongest frequency.
    2. The specific aspects of the physiology that it takes advantage of: 
        - We use PPG signals for measuring heart rate. 
        - The capillaries in the wrist are filled with blood when the ventricles contract.
        - When there are few red blood cells in these capillaries, the light from the PPG sensor will be reflected in a large amount. 
        - When the blood cells are many, they absorb the light, so the optical detector will detect less light.
        - Changes in the light detected by the optical detector, will produce an oscillating waveform which is the pulse rate.
    3. A describtion of the algorithm outputs: The algorithm outputs the estimated frequency (in BPM) along with its confidence score.
    4. Caveats on algorithm outputs: The algorithm can be affected by noise from other sources.
    5. Common failure modes: The algorithm may not perform well in other stages of changing heart rate due to arm and hand movements.
- **Algorithm Performance**
    - The performance is calculated using the mean absolute error between the estimations and the ground truth signal.
    - One can use other metrics like accuracy, recall, percision, f1-score.
    - The mean absolute error of the algorithm when evaluated on the test set is 8.77.
        
        

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 